In [ ]:


import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import time
import os

# PARAMETERS
IMG_SIZE = (128, 128)
BATCH_SIZE = 20
EPOCHS = 5

# Path to your dataset (change this)
DATA_DIR = '/home/mylove/4.1/AI lab /checked'

def dataset_loaders(data_dir):
    datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
    )

    train_gen = datagen.flow_from_directory(
        data_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training'
    )

    val_gen = datagen.flow_from_directory(
        data_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation'
    )

    return train_gen, val_gen

train_data, val_data = dataset_loaders(DATA_DIR)
class_names = list(train_data.class_indices.keys())
print("Detected Classes:", class_names)


Found 814 images belonging to 2 classes.
Found 203 images belonging to 2 classes.
Detected Classes: ['resized_images-sheuli', 'resized_images_togor']


In [16]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [17]:
# Input layer
inputs = Input(shape=(128, 128, 3), name="input_layer")

In [18]:
# ensure 'inputs' is defined (may be created in an earlier cell)
try:
	inputs
except NameError:
	inputs = Input(shape=(128, 128, 3), name="input_layer")

# -------- Block 1 --------
b1_c1 = Conv2D(64, (3, 3), activation='relu', padding='same', name="block1_conv1")(inputs)
b1_c2 = Conv2D(64, (3, 3), activation='relu', padding='same', name="block1_conv2")(b1_c1)
b1_p  = MaxPooling2D((2, 2), strides=(2, 2), name="block1_pool")(b1_c2)
# -------- Block 2 --------
# -------- Block 2 --------
b2_c1 = Conv2D(128, (3, 3), activation='relu', padding='same', name="block2_conv1")(b1_p)
b2_c2 = Conv2D(128, (3, 3), activation='relu', padding='same', name="block2_conv2")(b2_c1)
b2_p  = MaxPooling2D((2, 2), strides=(2, 2), name="block2_pool")(b2_c2)
# -------- Block 3 --------
# -------- Block 3 --------
b3_c1 = Conv2D(256, (3, 3), activation='relu', padding='same', name="block3_conv1")(b2_p)
b3_c2 = Conv2D(256, (3, 3), activation='relu', padding='same', name="block3_conv2")(b3_c1)
b3_c3 = Conv2D(256, (3, 3), activation='relu', padding='same', name="block3_conv3")(b3_c2)
b3_p  = MaxPooling2D((2, 2), strides=(2, 2), name="block3_pool")(b3_c3)
# -------- Block 4 --------
b4_c1 = Conv2D(512, (3, 3), activation='relu', padding='same', name="block4_conv1")(b3_p)
b4_c2 = Conv2D(512, (3, 3), activation='relu', padding='same', name="block4_conv2")(b4_c1)
b4_c3 = Conv2D(512, (3, 3), activation='relu', padding='same', name="block4_conv3")(b4_c2)
b4_p  = MaxPooling2D((2, 2), strides=(2, 2), name="block4_pool")(b4_c3)
# -------- Block 5 --------

b5_c1 = Conv2D(512, (3, 3), activation='relu', padding='same', name="block5_conv1")(b4_p)
b5_c2 = Conv2D(512, (3, 3), activation='relu', padding='same', name="block5_conv2")(b5_c1)
b5_c3 = Conv2D(512, (3, 3), activation='relu', padding='same', name="block5_conv3")(b5_c2)
b5_p  = MaxPooling2D((2, 2), strides=(2, 2), name="block5_pool")(b5_c3)


In [19]:
# -------- Fully Connected Layers --------
flat   = Flatten(name="flatten_layer")(b5_p)
fc1    = Dense(4096, activation='relu', name="fc1")(flat)
fc2    = Dense(4096, activation='relu', name="fc2")(fc1)
outputs = Dense(1000, activation='softmax', name="output_layer")(fc2)

In [20]:
# -------- Model Definition --------
model = Model(inputs=inputs, outputs=outputs, name="vgg16_style_model")
from tensorflow.keras.applications import VGG16

# Load the VGG16 model
# include_top=True  → include the 3 fully-connected layers at the top
# weights='imagenet' → load pretrained weights (set None to train from scratch)
# input_shape=(224, 224, 3) → standard VGG16 input size

model = VGG16(include_top=True, weights='imagenet', input_shape=(224, 224, 3))
model.summary(show_trainable=True)

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_2 (InputLayer)  │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block1_conv1 (Conv2D)       │ (None, 224, 224, 64)  │      1,792 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block1_conv2 (Conv2D)       │ (None, 224, 224, 64)  │     36,928 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block1_pool (MaxPooling2D)  │ (None, 112, 112, 64)  │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block2_conv1 (Conv2D)       │ (None, 112, 112, 128) │     73,856 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block2_conv2 (Conv2D)       │ (None, 112, 112, 128) │    147,584 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block2_pool (MaxPooling2D)  │ (None, 56, 56, 128)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block3_conv1 (Conv2D)       │ (None, 56, 56, 256)   │    295,168 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block3_conv2 (Conv2D)       │ (None, 56, 56, 256)   │    590,080 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block3_conv3 (Conv2D)       │ (None, 56, 56, 256)   │    590,080 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block3_pool (MaxPooling2D)  │ (None, 28, 28, 256)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block4_conv1 (Conv2D)       │ (None, 28, 28, 512)   │  1,180,160 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block4_conv2 (Conv2D)       │ (None, 28, 28, 512)   │  2,359,808 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block4_conv3 (Conv2D)       │ (None, 28, 28, 512)   │  2,359,808 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block4_pool (MaxPooling2D)  │ (None, 14, 14, 512)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block5_conv1 (Conv2D)       │ (None, 14, 14, 512)   │  2,359,808 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block5_conv2 (Conv2D)       │ (None, 14, 14, 512)   │  2,359,808 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block5_conv3 (Conv2D)       │ (None, 14, 14, 512)   │  2,359,808 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block5_pool (MaxPooling2D)  │ (None, 7, 7, 512)     │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ flatten (Flatten)           │ (None, 25088)         │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ fc1 (Dense)                 │ (None, 4096)          │ 102,764,5… │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ fc2 (Dense)                 │ (None, 4096)          │ 16,781,312 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ predictions (Dense)         │ (None, 1000)          │  4,097,000 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 138,357,544 (527.79 MB)

 Trainable params: 138,357,544 (527.79 MB)

 Non-trainable params: 0 (0.00 B)